In [1]:
from example.crypto_store import InMemCryptoStore
from eventsourcing.encryption import CryptoRepository
from eventsourcing.repositories import EventStoreRepository
from example.event_store import InMemEventStore
from example.user import User
from datetime import date
from example.guid import guid

### Initialize the in-memory encryption key store, eventstore, and repositories

In [2]:
crypto_store = InMemCryptoStore()
event_store = InMemEventStore()

CryptoRepository.crypto_store = crypto_store
user_repo = EventStoreRepository[User](event_store, User)

### Create a user and show that its attributes are clear

In [3]:
user = User(guid(), "Paul", "Boulanger", date(1997,2,18))

print(user.id, user.first_name, user.last_name, user.date_of_birth.isoformat())


b6fce6e5-94be-48ce-aa70-72afefc0654b Paul Boulanger 1997-02-18


### Save the user in the eventstore

The eventstore and key store are both empty

In [4]:
print(event_store.current)
print(crypto_store.store)

{}
{}


Saving the user in the eventstore

In [5]:
await user_repo.save(user,0)


The events related to the user are saved in the eventstore and an encryption key linked to the user was created and stored in the encryption key store

In [6]:
print(event_store.current)
print(crypto_store.store)

{'user-b6fce6e5-94be-48ce-aa70-72afefc0654b': [<example.event_store.EventDescriptor object at 0x000001B118E50D10>]}
{'b6fce6e5-94be-48ce-aa70-72afefc0654b': b'63iRxnUE3twaH7x9uougMkM6oDkdMDIV2tlvNz6-uSw='}


### Showing the encryption

The personnal data (in this case: `first_name`, `last_name`, `month_of_birth` and `day_of_birth`) are encrypted in the eventstore

In [7]:
event_store.current[User.to_stream_id(user.id)][0].event_data

'{"id": "b6fce6e5-94be-48ce-aa70-72afefc0654b", "first_name": "gAAAAABmjiB0T72qsdLYnssnKHqVa52Tjk75QBDVLHFisVf_3U5lwIMbxECmvGcqtYVADiOANiVim503AmCx09zn-AONlyRiXw==", "last_name": "gAAAAABmjiB0t14drIpeLVXLnqkkrME56AKfT0dACPrwoqe8rjisK29SHRth7CG_Wf4_Z23ooSf0WW5GI7idlg9-H5_IAKImUg==", "year_of_birth": 1997, "month_of_birth": "gAAAAABmjiB0Gde4rZDSuJ_7Ze4rubW0VWPWXBHJGCyrKqbLbVsjhSj7-UZ8rjebGcUj6Wa4kzPI4BDXDwiDs1c9O6SiCUTi_A==", "day_of_birth": "gAAAAABmjiB0toTcX43G7B0fkrgiN--TBZnfyMLHFfZAwpBARyUe6UXjTMAxoI1FUQXvkaf173hUisn6babgUX7ookOCWUkHTg=="}'

Since there is an encryption key, we can retrieve the personnal informations of the user in a decrypted state

In [8]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='b6fce6e5-94be-48ce-aa70-72afefc0654b', first_name='Paul', last_name='Boulanger', year_of_birth=1997, month_of_birth=2, day_of_birth=18)]

We delete the user's encryption key

In [9]:
crypto_store.remove(user.id)

Now the personnal informations are still encrypted

In [10]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='b6fce6e5-94be-48ce-aa70-72afefc0654b', first_name='gAAAAABmjiB0T72qsdLYnssnKHqVa52Tjk75QBDVLHFisVf_3U5lwIMbxECmvGcqtYVADiOANiVim503AmCx09zn-AONlyRiXw==', last_name='gAAAAABmjiB0t14drIpeLVXLnqkkrME56AKfT0dACPrwoqe8rjisK29SHRth7CG_Wf4_Z23ooSf0WW5GI7idlg9-H5_IAKImUg==', year_of_birth=1997, month_of_birth='gAAAAABmjiB0Gde4rZDSuJ_7Ze4rubW0VWPWXBHJGCyrKqbLbVsjhSj7-UZ8rjebGcUj6Wa4kzPI4BDXDwiDs1c9O6SiCUTi_A==', day_of_birth='gAAAAABmjiB0toTcX43G7B0fkrgiN--TBZnfyMLHFfZAwpBARyUe6UXjTMAxoI1FUQXvkaf173hUisn6babgUX7ookOCWUkHTg==')]

When we load our user from the eventstore, the personnal information are encrypted or replaced by fake information

In [11]:
new_user = User()
new_user.loads_from_history(events)
print(new_user.id, new_user.first_name, new_user.last_name, new_user.date_of_birth.isoformat())

b6fce6e5-94be-48ce-aa70-72afefc0654b gAAAAABmjiB0T72qsdLYnssnKHqVa52Tjk75QBDVLHFisVf_3U5lwIMbxECmvGcqtYVADiOANiVim503AmCx09zn-AONlyRiXw== gAAAAABmjiB0t14drIpeLVXLnqkkrME56AKfT0dACPrwoqe8rjisK29SHRth7CG_Wf4_Z23ooSf0WW5GI7idlg9-H5_IAKImUg== 1997-01-01
